In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
data_path = '../data/'
test_round1_filepath = data_path + 'round1_test.csv'
train_round1_filepath = data_path + 'round1_train.csv'
item_attr_round1_filepath = data_path + 'round1_item_attr.csv'

In [ ]:
df_train = pd.read_csv(train_round1_filepath)

In [ ]:
df_test = pd.read_csv(test_round1_filepath)
df_item_attr = pd.read_csv(item_attr_round1_filepath)

In [ ]:
print("shape:test=%r, train=%r, item_attr=%r" %(df_test.shape, df_train.shape, df_item_attr.shape))

In [ ]:
df_train.head()

In [ ]:
df_item_attr.head()

### shu ju tan suo

In [ ]:
def onehot_column(df, column_name):
    mapping = {label: idx for idx,label in enumerate(set(df[column_name]))}
    df[column_name] = df[column_name].map(mapping)

In [ ]:
def get_item(df, id_str, id_num):
    return df.loc[df[id_str] == id_num]

In [ ]:
def get_buyer_admin_id_in2country(df):
    grp_mean = df.groupby('buyer_admin_id')['buyer_country_id'].agg(np.mean)
    buyer_in2country_admin_id = grp_mean[grp_mean != 1][grp_mean != 0].index
    return buyer_in2country_admin_id

In [ ]:
def drop_buyer_in2country(df):
    assert isinstance(df, pd.DataFrame)
    grp_mean = df.groupby('buyer_admin_id')['buyer_country_id'].agg(np.mean)
    buyer_in2country_admin_id = grp_mean[grp_mean != 1][grp_mean != 0].index
    buyer_in2country_index = df[df['buyer_admin_id'].isin(buyer_in2country_admin_id.to_list())].index.to_list()
    df.drop(index=buyer_in2country_index, inplace=True)

In [ ]:
def get_train_data(train, test):
    onehot_column(train, 'buyer_country_id')
    onehot_column(df_test, 'buyer_country_id')
    drop_buyer_in2country(train)
    drop_buyer_in2country(test)
    # to_datetime
    train['create_order_time'] = pd.to_datetime(train['create_order_time'])
    test['create_order_time'] = pd.to_datetime(test['create_order_time'])
    # sort
    train.sort_values(by=['buyer_admin_id','create_order_time'], inplace=True)
    test.sort_values(by=['buyer_admin_id', 'create_order_time'], inplace=True)
    return train, test

In [ ]:
# ceshi 
grp_train_admin_id = df_train.groupby(['buyer_admin_id', 'buyer_country_id'])
get_item(grp_train_admin_id.size().reset_index(), 'buyer_admin_id', 10773)

In [ ]:
df_train, df_test = get_train_data(df_train, df_test)

In [ ]:
get_item(df_train, 'buyer_admin_id', 1).sort_values(by='irank')

In [ ]:
get_item(df_test, 'buyer_admin_id', 152).sort_values(by='irank')

In [ ]:
df_test['buyer_admin_id'].value_counts().sort_index()

In [ ]:
get_item(df_test, 'buyer_admin_id', 282).sort_values(by='irank')